In [14]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt # 데이터 시각화
import seaborn as sns
import missingno as msno
import os
import re
import multiprocessing # 여러 개의 일꾼 (cpu)들에게 작업을 분산시키는 역할
from multiprocessing import Pool 
from functools import partial # 함수가 받는 인자들 중 몇개를 고정 시켜서 새롭게 파생된 함수를 형성하는 역할
from data_loader_v2 import data_loader_v2
import math
%matplotlib inline
plt.rcParams["figure.figsize"] = (13,10)

In [3]:
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Embedding
from keras.layers import Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
from keras.utils import np_utils
import keras
import lightgbm as lgb
seed = 5

Using TensorFlow backend.


In [4]:
realData1 = "Train/30.csv"
realData2 = "Test/1154.csv"
realData3 = "Test/1168.csv"
realData4 = "additinal_data/additinal_data1"
realData5 = "additinal_data/additinal_data2"

In [5]:
train_folder = 'train/'
test_folder = 'test/'
train_label_path = 'train_label.csv'

In [6]:
train_list = os.listdir(train_folder)
test_list = os.listdir(test_folder)
train_label = pd.read_csv(train_label_path, index_col=0)

In [11]:
# 모든 csv 파일의 상태_B로 변화는 시점이 같다라고 가정
# 하지만, 개별 csv파일의 상태_B로 변화는 시점은 상이할 수 있음
def data_loader_all_v2(func, files, folder='', train_label=None, event_time=10, nrows=60):   
    func_fixed = partial(func, folder=folder, train_label=train_label, event_time=event_time, nrows=nrows)     
    if __name__ == '__main__':
        pool = Pool(processes=multiprocessing.cpu_count()) 
        df_list = list(pool.imap(func_fixed, files)) 
        pool.close()
        pool.join()        
    combined_df = pd.concat(df_list)    
    return combined_df

In [ ]:
train = data_loader_all_v2(data_loader_v2, train_list, folder=train_folder, train_label=train_label, event_time=10, nrows=60)
train.to_csv("train_v0.csv")
train = pd.read_csv("train.csv", index_col='Unnamed: 0')

test = data_loader_all_v2(data_loader_v2, test_list, folder=test_folder, train_label=None, event_time=10, nrows=60)
test.to_csv("test_v0.csv")
test = pd.read_csv("test.csv", index_col='Unnamed: 0')

In [ ]:
train_label = pd.read_csv("train_label.csv")
plt.hist(train_label['label'], bins=len(train_label['label'].unique()))
plt.xlim(0,200)
plt.ylim(bottom=0)
plt.xlabel('label')
plt.ylabel('# of label')
plt.show()

In [1]:
#train에서 각각의 라벨당 최소 545개 이상의 상태 B값이 있음
#유사도 측정을 통한 상태 A, B구분 연구

In [9]:
def cosine_similarity(data):
    lentgh = len(data)
    sim_list = []
    for i in range(length-1):
        sim_list.append(np.dot(data.iloc[i], data.iloc[i+1]) / (np.sqrt(np.dot(data.iloc[i], data.iloc[i])) * np.sqrt(np.dot(data.iloc[i+1], data.iloc[i+1]))))
    return sim_list    